In [83]:
import gym
import numpy as np

In [93]:
# 创建环境
env = gym.make('CliffWalking-v0',render_mode='human')

# 2 RIGHT

# 3 LEFT

In [90]:
np.random.choice([2,3])

np.int64(3)

In [94]:
env.reset()
state,_ = env.reset()
done = False
while not done:
    env.render()
    action = int(np.random.choice([2,3]))
    next_state, reward, done, truncated, _ = env.step(action)
    print(action,reward)

3 -1
3 -1
3 -1
3 -1
2 -1
3 -1
2 -1
2 -1
2 -1
2 -1
2 -1
3 -1
3 -1
3 -1
3 -1
3 -1
3 -1
3 -1
2 -1
2 -1
2 -1
3 -1
3 -1
3 -1
2 -1
3 -1


KeyboardInterrupt: 

In [4]:
env.reset()
env.render()

2025-04-22 15:22:26.151 python[80130:161978726] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-22 15:22:26.151 python[80130:161978726] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [5]:
env.close()

In [9]:
from torch import nn
import torch

In [18]:
class PolicyNet(nn.Module):
    
    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim,200)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(200,output_dim)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self,state):
        ### n
        x = self.linear1(state)
        x = self.relu(x)
        x = self.linear2(x) # n
        x = self.softmax(x) # n
        return x

In [24]:
from torch.distributions import Categorical
import numpy as np
np.bool8 = np.bool_

from torch.optim import AdamW

In [48]:
class Agent:
    
    def __init__(self):
        self.policy_net = PolicyNet(48,4)
        self.optimizer = AdamW(self.policy_net.parameters(),lr=1e-3)
    
    def sample_action(self,state):
        probs = self.policy_net(state) # 4
        if np.random.uniform() < 0.5:
            action = np.random.randint(0,4)
            return action, torch.log(probs[action]+1e-8)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action.item(),log_prob
    
    def update(self,rewards,log_probs):
        ### 一次游戏时间
        ret = []
        adding = 0
        for r in rewards[::-1]:
            adding = adding * 0.99 + r
            ret.insert(0,adding)
        ret = torch.FloatTensor(ret)
        ret = ret - ret.mean()
        ret = ret / (ret.std()+1e-8)
        
        r_log_probs = []
        for r,log_prob in zip(ret,log_probs):
            r_log_probs.append(-r*log_prob)
        r_log_probs = torch.vstack(r_log_probs)
        
        loss = r_log_probs.sum()
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss

In [49]:
def convert2tensor(state):
    state_arr = np.zeros(48)
    state_arr[state] = 1
    state_arr = torch.FloatTensor(state_arr)
    return state_arr

In [57]:
def train(agent,env):
    success_count = []
    max_size = 1000
    for epoch in range(20000):
        rewards = []
        log_probs = []
        terminated = False
        success = True
        state,_ = env.reset()
        while not terminated and len(log_probs) < max_size:
            state_arr = convert2tensor(state)
            action, log_prob = agent.sample_action(state_arr)
            next_state, reward, terminated, truncated, _ = env.step(action)
            state = next_state
            rewards.append(reward)
            log_probs.append(log_prob)
        
        loss = agent.update(rewards,log_probs) 
        success_count.append(success)
        
        
        if (epoch+1) % 10 == 0:
            print(f'success rate:  {len([s for s in success_count[-100:] if s]) / 100}')
            print(f'epoch: {epoch}, loss: {loss}, rewards: {sum(rewards)}, count: {len(rewards)}')

In [55]:
agent = Agent()



In [78]:
env = gym.make('CliffWalking-v0')
train(agent,env)

success rate:  0.1
epoch: 9, loss: 4.495227813720703, rewards: -14, count: 14
success rate:  0.2
epoch: 19, loss: 0.16866688430309296, rewards: -13, count: 13
success rate:  0.3
epoch: 29, loss: 0.18677257001399994, rewards: -13, count: 13
success rate:  0.39
epoch: 39, loss: 0.1880650818347931, rewards: -13, count: 13
success rate:  0.49
epoch: 49, loss: 0.1812109798192978, rewards: -13, count: 13
success rate:  0.59
epoch: 59, loss: 3.050615072250366, rewards: -15, count: 15
success rate:  0.69
epoch: 69, loss: 0.17841756343841553, rewards: -13, count: 13
success rate:  0.78
epoch: 79, loss: 0.18196885287761688, rewards: -13, count: 13
success rate:  0.88
epoch: 89, loss: 0.19397619366645813, rewards: -13, count: 13
success rate:  0.98
epoch: 99, loss: 0.19837002456188202, rewards: -13, count: 13
success rate:  0.98
epoch: 109, loss: 0.1943211406469345, rewards: -13, count: 13
success rate:  0.97
epoch: 119, loss: -14.634164810180664, rewards: -31, count: 22
success rate:  0.97
epoch

success rate:  0.97
epoch: 1049, loss: 0.27820929884910583, rewards: -13, count: 13
success rate:  0.97
epoch: 1059, loss: 4.648697376251221, rewards: -15, count: 15
success rate:  0.98
epoch: 1069, loss: 0.30566275119781494, rewards: -13, count: 13
success rate:  0.97
epoch: 1079, loss: 0.3154502809047699, rewards: -13, count: 13
success rate:  0.97
epoch: 1089, loss: 1.6023955345153809, rewards: -15, count: 15
success rate:  0.96
epoch: 1099, loss: 0.3120190501213074, rewards: -13, count: 13
success rate:  0.97
epoch: 1109, loss: 3.681757688522339, rewards: -14, count: 14
success rate:  0.98
epoch: 1119, loss: 0.3035407066345215, rewards: -13, count: 13
success rate:  0.98
epoch: 1129, loss: 0.2907388508319855, rewards: -13, count: 13
success rate:  0.98
epoch: 1139, loss: 0.2735762596130371, rewards: -13, count: 13
success rate:  0.98
epoch: 1149, loss: 0.2581138610839844, rewards: -13, count: 13
success rate:  0.98
epoch: 1159, loss: 3.695385456085205, rewards: -14, count: 14
succe

success rate:  0.98
epoch: 2089, loss: 4.978102684020996, rewards: -15, count: 15
success rate:  0.98
epoch: 2099, loss: -23.766298294067383, rewards: -14, count: 14
success rate:  0.98
epoch: 2109, loss: 0.1376989185810089, rewards: -13, count: 13
success rate:  0.99
epoch: 2119, loss: 0.1336345672607422, rewards: -13, count: 13
success rate:  0.98
epoch: 2129, loss: 0.13146424293518066, rewards: -13, count: 13
success rate:  0.99
epoch: 2139, loss: 0.13025225698947906, rewards: -13, count: 13
success rate:  0.99
epoch: 2149, loss: 0.13968908786773682, rewards: -13, count: 13
success rate:  0.99
epoch: 2159, loss: 0.1523973047733307, rewards: -13, count: 13
success rate:  0.98
epoch: 2169, loss: 0.16885533928871155, rewards: -13, count: 13
success rate:  0.98
epoch: 2179, loss: 0.17491452395915985, rewards: -13, count: 13
success rate:  0.97
epoch: 2189, loss: -10.435534477233887, rewards: -34, count: 25
success rate:  0.97
epoch: 2199, loss: 0.19204701483249664, rewards: -13, count: 

success rate:  0.99
epoch: 3119, loss: 0.0849672257900238, rewards: -13, count: 13
success rate:  0.99
epoch: 3129, loss: 0.08404257148504257, rewards: -13, count: 13
success rate:  0.99
epoch: 3139, loss: 0.08154365420341492, rewards: -13, count: 13
success rate:  0.99
epoch: 3149, loss: 0.08165095001459122, rewards: -13, count: 13
success rate:  0.99
epoch: 3159, loss: 4.634336948394775, rewards: -14, count: 14
success rate:  0.99
epoch: 3169, loss: 0.08478328585624695, rewards: -13, count: 13
success rate:  0.98
epoch: 3179, loss: 0.08909983932971954, rewards: -13, count: 13
success rate:  0.98
epoch: 3189, loss: 0.0916585624217987, rewards: -13, count: 13
success rate:  0.98
epoch: 3199, loss: 0.09279583394527435, rewards: -13, count: 13
success rate:  0.98
epoch: 3209, loss: -0.28948846459388733, rewards: -15, count: 15
success rate:  0.98
epoch: 3219, loss: 0.09735463559627533, rewards: -13, count: 13
success rate:  0.98
epoch: 3229, loss: 0.1034163385629654, rewards: -13, count:

success rate:  0.98
epoch: 4099, loss: 0.05983216315507889, rewards: -13, count: 13
success rate:  0.98
epoch: 4109, loss: 0.058105938136577606, rewards: -13, count: 13
success rate:  0.97
epoch: 4119, loss: 0.05546220391988754, rewards: -13, count: 13
success rate:  0.96
epoch: 4129, loss: 0.05270032957196236, rewards: -13, count: 13
success rate:  0.95
epoch: 4139, loss: 0.052141882479190826, rewards: -13, count: 13
success rate:  0.95
epoch: 4149, loss: 0.05073413997888565, rewards: -13, count: 13
success rate:  0.95
epoch: 4159, loss: 0.04947367310523987, rewards: -13, count: 13
success rate:  0.95
epoch: 4169, loss: 0.05110994353890419, rewards: -13, count: 13
success rate:  0.96
epoch: 4179, loss: 0.054799553006887436, rewards: -13, count: 13
success rate:  0.95
epoch: 4189, loss: 0.055542368441820145, rewards: -13, count: 13
success rate:  0.95
epoch: 4199, loss: 0.053672969341278076, rewards: -13, count: 13
success rate:  0.96
epoch: 4209, loss: 0.052312158048152924, rewards: -

...

success rate:  0.97
epoch: 14229, loss: 0.11422033607959747, rewards: -13, count: 13
success rate:  0.96
epoch: 14239, loss: 0.11152425408363342, rewards: -13, count: 13
success rate:  0.96
epoch: 14249, loss: 0.11968022584915161, rewards: -13, count: 13
success rate:  0.96
epoch: 14259, loss: 0.12599648535251617, rewards: -13, count: 13
success rate:  0.96
epoch: 14269, loss: 0.12741896510124207, rewards: -13, count: 13
success rate:  0.98
epoch: 14279, loss: 0.13251759111881256, rewards: -13, count: 13
success rate:  0.98
epoch: 14289, loss: 0.13745033740997314, rewards: -13, count: 13
success rate:  0.98
epoch: 14299, loss: 0.12762022018432617, rewards: -13, count: 13
success rate:  0.98
epoch: 14309, loss: 0.10992453247308731, rewards: -13, count: 13
success rate:  0.97
epoch: 14319, loss: 0.09361094981431961, rewards: -13, count: 13
success rate:  0.96
epoch: 14329, loss: 0.08627571910619736, rewards: -13, count: 13
success rate:  0.96
epoch: 14339, loss: 0.08289732038974762, rewa

success rate:  0.98
epoch: 15229, loss: 0.019260529428720474, rewards: -13, count: 13
success rate:  0.98
epoch: 15239, loss: 0.01867314800620079, rewards: -13, count: 13
success rate:  0.98
epoch: 15249, loss: 0.01859157718718052, rewards: -13, count: 13
success rate:  0.98
epoch: 15259, loss: 0.018376117572188377, rewards: -13, count: 13
success rate:  0.97
epoch: 15269, loss: 0.017226092517375946, rewards: -13, count: 13
success rate:  0.97
epoch: 15279, loss: 0.016491897404193878, rewards: -13, count: 13
success rate:  0.96
epoch: 15289, loss: 0.016345467418432236, rewards: -13, count: 13
success rate:  0.96
epoch: 15299, loss: 7.784084796905518, rewards: -15, count: 15
success rate:  0.96
epoch: 15309, loss: -26.763919830322266, rewards: -26, count: 17
success rate:  0.96
epoch: 15319, loss: 0.01982617750763893, rewards: -13, count: 13
success rate:  0.97
epoch: 15329, loss: 0.019765270873904228, rewards: -13, count: 13
success rate:  0.97
epoch: 15339, loss: 0.019350040704011917,

success rate:  0.98
epoch: 16239, loss: 0.02047327719628811, rewards: -13, count: 13
success rate:  0.98
epoch: 16249, loss: 0.019620412960648537, rewards: -13, count: 13
success rate:  0.98
epoch: 16259, loss: 0.018833070993423462, rewards: -13, count: 13
success rate:  0.98
epoch: 16269, loss: 0.018108662217855453, rewards: -13, count: 13
success rate:  0.99
epoch: 16279, loss: 0.017440522089600563, rewards: -13, count: 13
success rate:  1.0
epoch: 16289, loss: 0.01682206057012081, rewards: -13, count: 13
success rate:  1.0
epoch: 16299, loss: 0.016246693208813667, rewards: -13, count: 13
success rate:  0.99
epoch: 16309, loss: 0.01571212150156498, rewards: -13, count: 13
success rate:  0.99
epoch: 16319, loss: 0.015453032217919827, rewards: -13, count: 13
success rate:  0.99
epoch: 16329, loss: 0.01804491877555847, rewards: -13, count: 13
success rate:  0.99
epoch: 16339, loss: 0.019547777250409126, rewards: -13, count: 13
success rate:  0.99
epoch: 16349, loss: 0.019570916891098022

success rate:  0.97
epoch: 17249, loss: 0.059064604341983795, rewards: -13, count: 13
success rate:  0.97
epoch: 17259, loss: 0.05094776302576065, rewards: -13, count: 13
success rate:  0.97
epoch: 17269, loss: 1.5033810138702393, rewards: -15, count: 15
success rate:  0.98
epoch: 17279, loss: 0.05755005031824112, rewards: -13, count: 13
success rate:  1.0
epoch: 17289, loss: 0.06079469621181488, rewards: -13, count: 13
success rate:  1.0
epoch: 17299, loss: -15.907127380371094, rewards: -17, count: 17
success rate:  1.0
epoch: 17309, loss: 0.06184326112270355, rewards: -13, count: 13
success rate:  1.0
epoch: 17319, loss: 0.061656296253204346, rewards: -13, count: 13
success rate:  1.0
epoch: 17329, loss: 0.06965215504169464, rewards: -13, count: 13
success rate:  1.0
epoch: 17339, loss: 0.06934206187725067, rewards: -13, count: 13
success rate:  1.0
epoch: 17349, loss: 0.0642440915107727, rewards: -13, count: 13
success rate:  0.98
epoch: 17359, loss: -16.45240592956543, rewards: -15

KeyboardInterrupt: 

In [77]:
def sample_action(self,state):
    probs = self.policy_net(state) # 4
    if np.random.uniform() < 0.01:
        action = np.random.randint(0,4)
        return action, torch.log(probs[action]+1e-8)
    dist = Categorical(probs)
    action = dist.sample()
    log_prob = dist.log_prob(action)
    return action.item(),log_prob

# 替换方法
import types
agent.sample_action = types.MethodType(sample_action, agent)

In [80]:
import time
def visualize_agent(env, agent, num_episodes=5):
    """
    渲染显示智能体的行动
    """
    env = gym.make('CliffWalking-v0', render_mode='human')  # 创建可视化环境
    
    for episode in range(num_episodes):
        state_tuple = env.reset()
        state = state_tuple[0] if isinstance(state_tuple, tuple) else state_tuple
        total_reward = 0
        steps = 0
        done = False
        
        print(f"\nEpisode {episode + 1}")
        
        while not done:
            env.render()  # 渲染当前状态
            
            # 将状态转换为one-hot编码
            state_onehot = np.zeros(48)
            state_onehot[state] = 1
            
            # 使用训练好的策略选择动作
            with torch.no_grad():
                if np.random.random() < 0.0:
                    action = np.random.randint(0, 4)
                else:
                    state_tensor = torch.FloatTensor(state_onehot)
                    probs = agent.policy_net(state_tensor)
                    action = probs.argmax().item()  # 使用最可能的动作
            
            # 执行动作
            step_result = env.step(action)
            if len(step_result) == 4:
                next_state, reward, done, _ = step_result
            else:
                next_state, reward, terminated, truncated, _ = step_result
                done = terminated or truncated
            
            total_reward += reward
            steps += 1
            state = next_state
            
            # 添加小延迟使动作更容易观察
            time.sleep(0.5)
        
        print(f"Episode finished after {steps} steps. Total reward: {total_reward}")
    
    env.close()

# 在主程序最后添加：
if __name__ == "__main__":    
    # 训练完成后显示智能体行动
    print("\nVisualizing trained agent behavior...")
    env = gym.make('CliffWalking-v0',render_mode='human')
    visualize_agent(env, agent)


Visualizing trained agent behavior...

Episode 1
Episode finished after 13 steps. Total reward: -13

Episode 2
Episode finished after 13 steps. Total reward: -13

Episode 3
Episode finished after 13 steps. Total reward: -13

Episode 4
Episode finished after 13 steps. Total reward: -13

Episode 5
Episode finished after 13 steps. Total reward: -13


In [81]:
env.close()